# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# Создание цикла для определения уникальных значений каждого столбца, чтобы увидеть все возможные значения.
for column in data.columns:
    print('Уникальные значения в колонке:',column)
    print(data[column].unique())
    print()

Уникальные значения в колонке: children
[ 1  0  3  2 -1  4 20  5]

Уникальные значения в колонке: days_employed
[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]

Уникальные значения в колонке: dob_years
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

Уникальные значения в колонке: education
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Уникальные значения в колонке: education_id
[0 1 2 3 4]

Уникальные значения в колонке: family_status
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Уникальные значения в колонке: family_status_id
[0 1 2 3 4]

Уникальные значения в колонке: gender
['F' 'M' 'XNA']

Ун

In [5]:
# Определение количества аномальных значений в столбце children
print('Количество строк, где детей 20:',data[data['children'] == 20].count())
print('Количество строк, где детей -1:',data[data['children'] == -1].count())

Количество строк, где детей 20: children            76
days_employed       67
dob_years           76
education           76
education_id        76
family_status       76
family_status_id    76
gender              76
income_type         76
debt                76
total_income        67
purpose             76
dtype: int64
Количество строк, где детей -1: children            47
days_employed       44
dob_years           47
education           47
education_id        47
family_status       47
family_status_id    47
gender              47
income_type         47
debt                47
total_income        44
purpose             47
dtype: int64


In [6]:
#Строка  со странным полом 'XNA'
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [7]:
#Подсчет количества строк с нулевым значением возраста
data[data['dob_years'] == 0].count()

children            101
days_employed        91
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income         91
purpose             101
dtype: int64

**Вывод**

В таблице двенадцать столбцов. Тип данных в столбцах — `object`,`int64`, `float64`.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — идентификатор семейного положения;
* `purpose` — цель получения кредита.

Количество значений в столбцах различается - есть пропущенные значения.
В семи колонках есть над чем поработать:
1. `children` - отрицательное значение -1, повторяющееся 47; значение 20, потворяющееся 76 раз.
2. `days_employed` - пропуски, отрицательные значения, и аномально высокие значения, например стаж 340000 дней(930 лет)!!
3. `dob_years` - 0 лет 101 человеку.
4. `education` - проблема с регистром.
5. `gender` - один человек с неизвестным полом (оставлю его/ее как есть).
6. `total_income` — пропуски.
7. `purpose` - много совпадающих по смыслу значений, такие как 'покупка жилья', 'покупка жилья для семьи', 'покупка жилой недвижимости' и т.д.

## Предобработка данных

### Обработка пропусков и ошибочных значений


В столбце `children` `-1` повторяется 47 раз, и `20` повторяется 76 раз.
Учитывая количество повторений, минус скорее всего означает тире. А ноль вполне мог случайно набираться, т.к. на клавиатуре он стоит рядом с двойкой.

In [8]:
# Преобразование отрицательных значений в положительные в столбце children
data['children'] = data['children'].apply(abs)

In [9]:
# преобразование 20 в 2
data.loc[data['children'] == 20, 'children'] = data.loc[data['children'] == 20, 'children']/10
data['children'].unique()
# на выходе получили тип float, чуть позже преобразуем  в int

array([1., 0., 3., 2., 4., 5.])

В столбце `days_employed` отрицательные значения скорее всего появились из-за неправильного ввода данных  и минус, просто-напросто, означает тире.

А вот высокие значения, например 340266, возможно исчисляются не в днях, а предположим в часах. 
Проверим максимальное значение стажа:

In [10]:
print('Максимальный стаж в годах:',data['days_employed'].max()/24/365)

Максимальный стаж в годах: 45.8624886387363


Почти 46 лет стажа ого-го!! Но главное цифра вполне реальная. Исправляем значения в колонке `days_employed` с часов на дни:

In [11]:
# преобразование отрицательных значений в положительные
data['days_employed'] = data['days_employed'].apply(abs)
# значения, которые больше 30000(чуть больше 82 лет) делим на 24 часа
data.loc[data['days_employed'] >30000, 'days_employed'] = data.loc[data['days_employed'] >30000, 'days_employed'] / 24

**Теперь перейдем к пропускам.**

Eсть две колонки с пропусками NaN: `days_employed` и `total_income`.
Более 2000 значений по доходу и стажу отсутствуют. Судя по огромному количеству ошибка имеет технический характер. Возможно данные были потеряны при обьединении и загрузке файлов. Нужно обратиться к разработчикам.

Пустые значения стажа и дохода логичнее заполнить медианными значениями по категориям занятости.
Для этого создадим функцию, где аргументом будет столбец.


In [12]:
# Функция для замены пропусков в колонках медианными значениеми по категориям занятости 'income_type'
def replace_pass(column):
    
    print('пропусков до работы функции:', data[column].isna().sum())
    # Цикл по категориям income_type
    for income in data['income_type'].unique():
        # Подсчет медианы для категории income_type
        median = data.loc[data['income_type'] == income, column].median()
        print(income,median)
        # Замена NaN на медиану
        data.loc[(data[column].isna())&(data['income_type'] == income), column] = median
    
    print('Пропусков после работы функции:', data[column].isna().sum())
    
    return

Применим функцию к столбцу `days_employed`

In [13]:
replace_pass('days_employed')

пропусков до работы функции: 2174
сотрудник 1574.2028211070854
пенсионер 15217.221094405468
компаньон 1547.3822226779334
госслужащий 2689.3683533043886
безработный 15267.235531008522
предприниматель 520.8480834953765
студент 578.7515535382181
в декрете 3296.7599620220594
Пропусков после работы функции: 0


Применим функцию к столбцу `total_income`

In [14]:
replace_pass('total_income')

пропусков до работы функции: 2174
сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
Пропусков после работы функции: 0


В столбце `dob_years` 101 значение равно 0. Это тоже можно считать пропусками. Но характер данной ошибки скорее человеческий фактор.

В данном случае можно заменить нули медианным значением по категориям занятости, как и выше. Единственная преграда: здесь нули, а не пропуски.

Заменим нули на пропуски NaN - для это потребуется применить метод `replace()`. После используем функцию `replace_pass`, созданную выше.

In [15]:
data['dob_years'] = data['dob_years'].replace(0, np.nan)

replace_pass('dob_years')

пропусков до работы функции: 101
сотрудник 39.0
пенсионер 60.0
компаньон 39.0
госслужащий 40.0
безработный 38.0
предприниматель 42.5
студент 22.0
в декрете 39.0
Пропусков после работы функции: 0


In [16]:
#Проверяем таблицу после обработки пропусков
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  float64
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(4), int64(3), object(5)
memory usage: 2.0+ MB


### Замена типа данных

После обработки пропусков в столбцах `children` и `dob_years` изменился тип данных с int на float. Необходимо вернуть тип данных int обратно.

`days_employed` указан в днях, зачем дробные значения дня не понятно. Поэтому преобразуем в целочисленный тип данных и этот столбец.

`total_income` указан в рублях. 1/10000 копейки да и копейки мало повлияют на исследовние. Заменим на int.

In [17]:
#Создание строки, с названиями столбцов
columns = ['children', 'days_employed', 'dob_years', 'total_income'] 
#Цикл проходящийся по нужным столбцам и заменяющий тип данных с float на int, методом astype()
for column in columns:
    data[column] = data[column].astype('int')
#Проверяем
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Обработка дубликатов

Для поиска дубликатов используем метод `duplicated()`, и  метод `sum()` для подсчета количества дубликатов.
Однако, в `education` есть проблемы с регистром - это может помешать найти все дубликаты.
Для удаления дубликатов используем метод `drop_dublicates()`, и метод `reset_index()` для обновления индексации.
Проверим на практике:

In [18]:
print('До изменения регистра:', data.duplicated().sum())
data['education'] = data['education'].str.lower()
print('После изменения регистра:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)
print('После удаления дубликатов:', data.duplicated().sum())

До изменения регистра: 54
После изменения регистра: 71
После удаления дубликатов: 0


Дубликатов немного - возможно кто-то оправлял заявку на кредит повторно.

### Лемматизация

Столбец `purpose` имеет много совпадающих по смыслу значений, такие как 'покупка жилья', 'покупка жилья для семьи', 'покупка жилой недвижимости' и т.д.

Здесь необходима лемматизация: приведем существительные в именительный падеж, глаголы в инфинитив. 

In [19]:
m = Mystem()
# Находим уникалльные значения в столбце purpose
text = data['purpose'].unique()
# склеиваем список в строку с помощью метода join() и лемматизируем строку
lemmas = m.lemmatize(' '.join(text))
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

Подсчитаем количество упоминаний каждого слова в тексте с помощью контейнера `Counter` из модуля `collections`.

In [20]:
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Итак, выделим самые частые существительные, отвечающие на смысл цели:

    1. жилье, недвижимость
    2. автомобиль
    3. образование
    4. свадьба
    
Напишем функцию, которая лемматизирует каждую ячейку столбца `purpose` и заменяет ее на обобщенную версию категории.

In [21]:
def category_purpose(row):
    #Лемматизируем ячейку
    purpose = row['purpose']
    lemmas = ' '.join(m.lemmatize(purpose))
    
    #условия после которых функция возвращает обобщенную версию категории
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    return 'цель не определена'

# Метод apply() применяем к датафрейму data;
# По умолчанию Pandas передаёт в функцию category_purpose() столбец.
# Чтобы на вход в функцию отправлялись строки, нужно указать параметр axis = 1 метода apply().
data['purpose'] = data.apply(category_purpose, axis=1)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


### Категоризация данных

Для того, чтобы ответить на вопрос зависимости дохода и возврата кредита в срок,
осталось категоризовать один столбец `total_income`.
Остальные столбцы приведены в порядок.

Для начала оценим данные `total_income`: выявим максимум, минимум, среднее и медиану.

In [22]:
print('Максимальный доход:',data['total_income'].max())
print('Минимальный доход:',data['total_income'].min())
print('Средне-арифметическое значение дохода:',data['total_income'].mean())
print('Медианное значение дохода',data['total_income'].median())

Максимальный доход: 2265604
Минимальный доход: 20667
Средне-арифметическое значение дохода: 165319.57229421087
Медианное значение дохода 142594.0


### По этим даным разобьем категории(достаточно условно):
   * низкий уровень дохода: до 100к,
   * средний уровень дохода: 100-200к,
   * высокий уровень дохода: 200-500к,
   * очень высокий уровень дохода : свыше 500к.
   
Напишем функцию категоризации по уровню дохода:

In [23]:
def total_income_group(row):
    total_income = row['total_income']
    
    # Условия при которых функция возвращает нужную категорию
    if total_income < 100000:
        return 'низкий уровень дохода'
    if total_income < 200000:
        return 'средний уровень дохода'
    if total_income < 500000:
        return 'высокий уровень дохода'
    
    return 'очень высокий уровень дохода'

# Метод apply() применяем к датафрейму data;
# По умолчанию Pandas передаёт в функцию total_income_group() столбец.
# Чтобы на вход в функцию отправлялись строки, нужно указать параметр axis = 1 метода apply().
data['total_income_group'] = data.apply(total_income_group, axis=1)
# подсчет количетсва человек в категории методом value_counts()
data['total_income_group'].value_counts()

средний уровень дохода          11924
высокий уровень дохода           4845
низкий уровень дохода            4463
очень высокий уровень дохода      222
Name: total_income_group, dtype: int64

Получилось вполне реалистично: богатых ~1% от общего количества,  бедных ~21%.

## Вопросы

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Чтобы ответить на этот и другие вопросы: создадим функцию с аргументом названия столбца, которая создает сводную таблицу.

В таблице будет видно категории введенного столбца, количество невозвращенных кредитов в этой категории, общее количество категории, процент невозвращенных кредитов.

In [24]:
def report(category):
    #создание сводной таблицы методом pivot_table()
    pt = data.pivot_table(index = [category], values = ['debt'], aggfunc = ['sum','count','mean'])
    #колонки в сводной таблице
    pt.columns = ['debt','total','%']
    #сортировка по столбцу total по убыванию методом sort_values()
    pt = pt.sort_values(by = 'total', ascending = False)
    #преобразование значения в столбце '%' методом округления round() и умножения на 100 
    pt['%'] = pt['%'].round(4)*100
    return pt


In [25]:
report('children')

,debt,total,%
children,,,
0,1063,14091,7.54
1,445,4855,9.17
2,202,2128,9.49
3,27,330,8.18
4,4,41,9.76
5,0,9,0.00


**Вывод**

* Зависимость есть. Люди у которых совсем нет детей реже имеют просрочку.
* Чем больше детей, тем меньше возвращают кредиты. 
* Исключение - в семье три ребенка, но данных в этой категории мало.

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [26]:
report('family_status')

,debt,total,%
family_status,,,
женат / замужем,931,12339,7.55
гражданский брак,388,4151,9.35
Не женат / не замужем,274,2810,9.75
в разводе,85,1195,7.11
вдовец / вдова,63,959,6.57


**Вывод**

* Связанные законным браком, люди больше отдают кредиты, нежели, живущие в гражданском браке.
* Не женатые/ не замужние люди хуже отдают кредиты, чем в разводе/вдовцы/вдовы.
* Зависимость есть: люди которые связаны законным браком, или были связаны законным браком отдают кредиты лучше,чем остальные.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [27]:
report('total_income_group')

,debt,total,%
total_income_group,,,
средний уровень дохода,1029,11924,8.63
высокий уровень дохода,344,4845,7.10
низкий уровень дохода,354,4463,7.93
очень высокий уровень дохода,14,222,6.31


**Вывод**

* Завимость почти прямая: чем больше доход тем меньше невозвратов. 
* Категории 'средний' и 'низкий' близки по невозвратам.

**- Как разные цели кредита влияют на его возврат в срок?**

In [28]:
report('purpose')

,debt,total,%
purpose,,,
недвижимость,782,10811,7.23
автомобиль,403,4306,9.36
образование,370,4013,9.22
свадьба,186,2324,8.00


**Вывод**

* При покупке недвижимости самый высокий процент возвращения.
* Менее надежные заемщики, покупающие автомобиль или оплачивающие образование.
* Решившие сыграть свадьбу, где-то посередине.

## Общий вывод

* При проведении исследования найдены прямые зависимости. 
* Можно обрисовать идеальных заемщиков: женатые/замужние или бывшие в законном браке, без детей, с высоким уровнем дохода, берущие кредит с целью купить недвижимость.